In [4]:
### compositionevo.ipynb
### Script to extract composition correlation evolution data

import csv
from importlib import reload
import numpy as npy
import pkdgrav_ss as pkd
import os
import copy
import math

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

mearth = 1./330060.

maxnumf = 6000   # max number of output steps
nsim = 21  # number of similar planetesimals to extract

#path = '../../../LocalData/'
path = 'data/'
loc = path + '022GTJf6hgas/'

emb = [0,1,2,3,4,5,6,7,8]  # embryo number to extract


In [2]:
# create output file list
stepfac = 1.
param = pkd.readparam(loc+'ss.par')
ndig = param['nDigits']
mstep = int(param['iOutInterval'])

k = 1
while int(param['nSteps']/param['iOutInterval']/k)>maxnumf:
    k += 1 # *=2
mstep *= int(k)

files = npy.arange(0,param['nSteps'] + mstep,stepfac*mstep).astype(int)#/param['iOutInterval']).astype(int) * int(param['iOutInterval'])
if files[-1]>param['nSteps']:
    files = files[0:-1]
if files[-1] != int(param['nSteps']):
    files = npy.append(files,int(param['nSteps']))

print(files,len(files))

amin = param['dDustBinsInner']
amax = param['dDustBinsOuter']
nbins = int(param['nDustBins'])
binedge = npy.linspace(amin,amax,nbins+1)

print(binedge)

[        0     20000     40000 ... 119960000 119980000 120000000] 6001
[0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2
 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3. ]


In [3]:
#reload(pkd)
col = 0
col = pkd.collisions(path=loc)

ss = pkd.ss()
ss.read(loc+'ss.{:0>10d}'.format(int(param['nSteps'])),extras=True)
ss.calcOE()
sorder = npy.argsort(ss.m)[::-1]
ind = npy.arange(len(ss.a))[(ss.a[sorder]<3.5)*(ss.e[sorder]<1.)]
indlen = len(ind)

sorder = npy.argsort(ss.m)[::-1]
id1 = ss.id[sorder]
ob = ss.origin[sorder]

for j in emb:
    emb = id1[ind[j]]  # pkdgrav embryo id
    print('embryo id: ',emb)
    obbig = ob[ind[j]]

    time=npy.array([])

    cct=npy.array([])
    cclist = [] #[cct,cct1,cct2,cct3,cct4,cct5,cct6,cct7,cct8]

    for l in range(nsim):
        cclist.append(cct.copy())

    mt = npy.array([])
    mlist = [] 

    for l in range(nsim):
        mlist.append(mt.copy())

    at = npy.array([])
    alist = [] 

    for l in range(nsim):
        alist.append(at.copy())


    trackIDk = npy.zeros(nsim)
    newtrackIDk = npy.zeros(nsim)
    changetk = npy.zeros(nsim)
    changet = 0.
    c0=npy.array([]) #=c1=c2=c3=c4=c5=c6=c7=c8=
    ck = [] #[c0,c1,c2,c3,c4,c5,c6,c7,c8]

    for l in range(nsim):
        ck.append(c0.copy())

    trackID = id1[ind[j]]
    c = npy.sort(col.get_history(trackID))

    print(c,npy.sort(c))

    print(col.t[c])


    resarr = npy.zeros(indlen)

    for i in range(0,indlen,1):
      res = npy.corrcoef(obbig,ob[ind[i]])
      resarr[i]=res[0,1]
    corder=npy.argsort(resarr)[::-1]


    # find final 'historical' ids of this embryo

    for ic in c[::-1]:
            if col.idlr[ic]==trackID:
                if col.idt[ic]!=trackID:
                    newtrackID = col.idt[ic]
                    changet = col.t[ic]
                    break
            elif col.idslr[ic] == trackID:
                if col.idp[ic]!=trackID:
                    newtrackID = col.idp[ic]
                    changet = col.t[ic]
                    break

    print('change:',changet,trackID,newtrackID,col.type[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic],col.idp[ic],col.idslr[ic],col.mslr[ic])

    # and similar planetesimals
    for k in range(1,nsim):
        trackIDk[k] = id1[ind[corder[k]]]
        ck[k] = npy.sort(col.get_history(trackIDk[k]))
        for ic in ck[k][::-1]:
            if col.idlr[ic]==trackIDk[k]:
                if col.idt[ic]!=trackIDk[k]:
                    newtrackIDk[k] = col.idt[ic]
                    changetk[k] = col.t[ic]
                    break
            elif col.idslr[ic]==trackIDk[k]:
                if col.idp[ic]!=trackIDk[k]:
                    newtrackIDk[k] = col.idp[ic]
                    changetk[k] = col.t[ic]
                    break


    # calculate correlations with embryo through time

    fout = open(loc+'comp_evolution-{:d}.dat'.format(j),'w')

    for fn in files[::-1]:
        sst = pkd.ss()
        if fn == 0:
            sst.read(loc+'ssic.ss')
            sst.id = sst.org_idx
            sst.origin = npy.array([npy.histogram(x,binedge)[0] for x in npy.sqrt(sst.x**2+sst.y**2)])
            #sst.origin = npy.array([npy.histogram(x,binedge[0:11])[0] for x in npy.sqrt(sst.x**2+sst.y**2)])
        else:
            sst.read(loc+'ss.{:>010d}'.format(fn),extras=True)
        sst.calcOE()
        time = npy.append(time,sst.header.t)

        while sst.header.t/2./math.pi < changet and newtrackID != trackID:
            trackID=newtrackID
            for ic in c[::-1][npy.argwhere((col.idlr[c[::-1]]==trackID)+(col.idslr[c[::-1]]==trackID))]:
                if col.idlr[ic]==trackID:
                    #if trackID== 58081 or trackID == 57006:
                    #    print(col.type[ic],col.t[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic])
                    if col.idt[ic]!=trackID and col.t[ic] < changet:
                        newtrackID = col.idt[ic]
                        changet = col.t[ic]
                        print('change:',changet,trackID,newtrackID,col.type[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic],col.idp[ic],col.idslr[ic],col.mslr[ic])
                        break
                elif col.idslr[ic]==trackID:
                    if col.idp[ic]!=trackID and col.t[ic] < changet:
                        newtrackID = col.idp[ic]
                        changet = col.t[ic]
                        print('change:',changet,trackID,newtrackID,col.type[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic],col.idp[ic],col.idslr[ic],col.mslr[ic])
                        break

        #print('trackID:',trackID)                
        #print(fn,sst.header.t,sst.m[sst.id==emb][0])
        res = npy.corrcoef(obbig,sst.origin[sst.id==trackID])
        #print res, len(sst.id[sst.id==trackID])
        cct = npy.append(cct,res[0,1])
        mt = npy.append(mt,sst.m[sst.id==trackID][0])
        at = npy.append(at,sst.a[sst.id==trackID][0])

        #write time mt cct
        #fout.write( '{:.10g}'.format(sst.header.t) + ' {:.8g}'.format(sst.m[sst.id==trackID][0]) + ' {:.4f}'.format(res[0,1]) )
        fout.write( '{:.10g}'.format(sst.header.t) + ' {:.8g}'.format(sst.m[sst.id==trackID][0]) + ' {:.4f}'.format(sst.a[sst.id==trackID][0]) + ' {:.4f}'.format(res[0,1]) )

        for k in range(1,nsim):
          while sst.header.t/2./math.pi < changetk[k] and newtrackIDk[k] != trackIDk[k]:
            trackIDk[k]=newtrackIDk[k]
            #ck[k] = col.get_history(trackIDk[k])
            for ic in ck[k][::-1][npy.argwhere((col.idlr[ck[k][::-1]]==trackIDk[k])+(col.idslr[ck[k][::-1]]==trackIDk[k]))]:
                if col.idlr[ic]==trackIDk[k]:
                    if col.idt[ic]!=trackIDk[k] and col.t[ic] < changetk[k]:
                        newtrackIDk[k] = col.idt[ic]
                        changetk[k] = col.t[ic]
                        print('change-k:',k,changetk[k],trackIDk[k],newtrackIDk[k],col.type[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic],col.idp[ic],col.idslr[ic],col.mslr[ic])
                        if k==1:
                            print(changetk[k],newtrackIDk[k])
                        break
                elif col.idslr[ic]==trackIDk[k]:
                    if col.idp[ic]!=trackIDk[k] and col.t[ic] < changetk[k]:
                        newtrackIDk[k] = col.idp[ic]
                        changetk[k] = col.t[ic]
                        print('change-k:',k,changetk[k],trackIDk[k],newtrackIDk[k],col.type[ic],col.idlr[ic],col.mlr[ic],col.idt[ic],col.mt[ic],col.idp[ic],col.idslr[ic],col.mslr[ic])
                        if k==1:
                            print(changetk[k],newtrackIDk[k])
                        break

          if len(sst.id[sst.id==trackIDk[k]])>0:
            res = npy.corrcoef(obbig,sst.origin[sst.id==trackIDk[k]])
            #print id1[ind[corder[k]]],res,len(sst.id[sst.id==id1[ind[corder[k]]]])
            cclist[k] = npy.append(cclist[k],res[0,1])
            mlist[k] = npy.append(mlist[k],sst.m[sst.id==trackIDk[k]][0])
            alist[k] = npy.append(alist[k],sst.a[sst.id==trackIDk[k]][0])
            # write mk cck
            #fout.write( ' {:.8g}'.format(sst.m[sst.id==trackIDk[k]][0]) + ' {:.4f}'.format(res[0,1]) )
            fout.write( ' {:.8g}'.format(sst.m[sst.id==trackIDk[k]][0]) +  ' {:.4f}'.format(sst.a[sst.id==trackIDk[k]][0]) + ' {:.4f}'.format(res[0,1]) )
          else:
            cclist[k] = npy.append(cclist[k],-99)
            mlist[k] = npy.append(mlist[k],1e-25)
            alist[k] = npy.append(alist[k],-99)
            # write mk cck
            #fout.write( ' {:.8g}'.format(1.e-25) + ' {:.4f}'.format(-99.0) )
            fout.write( ' {:.8g}'.format(1.e-25) + ' {:.4f}'.format(-99.0) + ' {:.4f}'.format(-99.0) )
        fout.write('\n')
    fout.close()

    print('File: comp_evolution-{:d}.dat'.format(j),' written.\n')

embryo id:  3506
Found 108 collisions for object: 3506
[  0   1   2   4   7  11  12  13  17  18  19  22  25  26  28  30  32  34
  37  38  42  45  46  47  52  56  57  58  59  60  61  62  64  65  66  70
  72  73  74  76  77  82  85  88  91  94  95  96  97  99 100 101 102 104
 105 106 107 108 109 112 116 126 129 131 134 139 145 148 149 150 152 154
 155 165 168 169 173 177 182 196 198 213 220 222 223 225 226 228 230 235
 247 248 250 251 252 256 258 259 263 265 267 270 272 274 283 297 300 312] [  0   1   2   4   7  11  12  13  17  18  19  22  25  26  28  30  32  34
  37  38  42  45  46  47  52  56  57  58  59  60  61  62  64  65  66  70
  72  73  74  76  77  82  85  88  91  94  95  96  97  99 100 101 102 104
 105 106 107 108 109 112 116 126 129 131 134 139 145 148 149 150 152 154
 155 165 168 169 173 177 182 196 198 213 220 222 223 225 226 228 230 235
 247 248 250 251 252 256 258 259 263 265 267 270 272 274 283 297 300 312]
[295194.24214349 295807.22740681 296960.54370193 297861.81825029
 3